In [314]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from dataloader import DataLoader
import numpy as np
from torchsummary import summary
import os 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Setup Data Loader

In [315]:
dataloader = DataLoader('/scratch/users/gmachi/codex/data/train/', batch_size=24, transfer=False)

In [316]:
USE_GPU = False
dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [317]:
files = os.listdir('/scratch/users/gmachi/codex/data/val/')
print('val')
set([fname.split('reg')[1].split('_')[0] for fname in files])

val


{'011', '016', '023', '030'}

In [318]:
files = os.listdir('/scratch/users/gmachi/codex/data/test/')
print('test')
set([fname.split('reg')[1].split('_')[0] for fname in files])

test


{'005', '006', '017', '019'}

In [319]:
files = os.listdir('/scratch/users/gmachi/codex/data/train/')
print('train')
completed = set([fname.split('reg')[1].split('_')[0] for fname in files])
print('completed:', completed)
remaining = set(['015','004','014','024','020','007','008','027','034','012']) - completed
print('remaining:', remaining)

train
completed: {'027', '020', '034', '008', '007', '015', '024', '014', '004', '012'}
remaining: set()


In [320]:
len(os.listdir('/scratch/users/gmachi/codex/data/train/'))

13069

# Training/Validating and Utility Functions

In [335]:
def train(model, optimizer, epochs=1):
    """
    Train a model on image data using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    
    print_every = 4
    val_every = 20
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    
    for e in range(epochs):
        for t, (fxy, x, y) in enumerate(dataloader):
            
            model.train()  # put model to training mode
            
            x = torch.from_numpy(x)
            y = torch.from_numpy(y)
            
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
            if t % print_every == 0:
                print('Checking validation accuracy:')
                check_train_accuracy(model)
            if t != 0 and t % val_every == 0:
                print('Checking validation accuracy:')
                check_accuracy(model)
                print()

In [336]:
def check_train_accuracy(model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    
    loader = DataLoader('/scratch/users/gmachi/codex/data/train/', batch_size=10, transfer=False)
    
    with torch.no_grad():
        for i, (fxy, x, y) in enumerate(loader):
            
            x = torch.from_numpy(x)
            y = torch.from_numpy(y)
            
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            
            scores = model(x)
            _, preds = scores.max(1)

            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
            if i > 5:
                break
            
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [337]:
def check_accuracy(model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    
    loader = DataLoader('/scratch/users/gmachi/codex/data/val/', batch_size=10, transfer=False)
    
    with torch.no_grad():
        for i, (x, y) in enumerate(loader):
            x = torch.from_numpy(x)
            y = torch.from_numpy(y)
            
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
            
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [338]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)
    
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    def forward(self, x):
        print(x.shape)
        return x

# Define Model

In [339]:
model = torchvision.models.vgg19(pretrained=False)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [340]:
# same structure as vgg19 model print(torchvision.models.vgg19(pretrained=False, progress = True))
model = nn.Sequential(
    
    nn.Conv2d(75, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), #64
    nn.ReLU(inplace = True),
    nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
    
    nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
    
    nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
    
    nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
    
    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace = True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
    
    Flatten(),
    
    nn.Linear(in_features= 4608, out_features=4096, bias=True),
    nn.ReLU(inplace = True),
    nn.Dropout(p=0.5),
    
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(inplace = True),
    nn.Dropout(p=0.5),
    
    nn.Linear(in_features=4096, out_features=2, bias=True),
    
)

In [341]:
# TO DEBUG MODEL:::
# add PrintLayer(), or use summary from torchsummary module:
# summary(your_model, input_size=(channels, H, W))
# https://towardsdatascience.com/model-summary-in-pytorch-b5a1e4b64d25

# Train Model

In [ ]:
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(),
                      lr=learning_rate,
                      momentum=0.9,
                      nesterov=True)

train(model, optimizer)